# TensorFlow/Keras로 구현하는 다층 퍼셉트론 (MLP)

**학습 목표:**
- 또 다른 주요 딥러닝 프레임워크인 **TensorFlow**와, 그 고수준 API인 **Keras**의 사용법을 익힙니다.
- Keras의 직관적인 `Sequential` API를 사용하여 층을 순서대로 쌓아 MLP 모델을 빠르고 쉽게 구축하는 방법을 학습합니다.
- 모델 `compile`, `fit`, `evaluate`의 3단계로 이루어지는 Keras의 표준적인 훈련 및 평가 워크플로우를 이해합니다.
- `history` 객체를 활용하여 훈련 과정의 손실 및 정확도를 시각화하고, 과적합 여부를 분석합니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import matplotlib.pyplot as plt

### (1) 하이퍼파라미터 설정

In [ ]:
batch_size = 64
epochs = 10

### (2) 데이터 준비: MNIST
`keras.datasets` 모듈을 사용하면 MNIST 데이터셋을 간편하게 불러올 수 있습니다. 신경망의 안정적인 학습을 위해 이미지 픽셀 값을 0과 1 사이로 정규화합니다.

In [ ]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 픽셀 값 정규화 (0-255 -> 0-1)
X_train, X_test = X_train / 255.0, X_test / 255.0

print(f"Train data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# 데이터 샘플 확인
plt.figure(figsize=(10,1))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X_train[i], cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.show()

### (3) 모델 정의: MLP (Sequential API)
Keras의 `Sequential` API는 층(Layer)을 리스트 형태로 전달하여 모델을 매우 간단하게 정의할 수 있게 해줍니다.
- `Flatten`: 28x28의 2D 이미지 데이터를 784개의 1D 벡터로 변환하여 Dense 층에 입력할 수 있도록 합니다.
- `Dense`: 완전 연결 계층(Fully-Connected Layer)입니다.
- `activation='relu'`: 은닉층의 활성화 함수로 ReLU를 사용합니다.
- `activation='softmax'`: 출력층의 활성화 함수로 Softmax를 사용하여 10개 클래스에 대한 확률 분포를 출력합니다.

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),    # 입력층
    Dense(512, activation='relu'),      # 첫 번째 은닉층
    Dense(256, activation='relu'),      # 두 번째 은닉층
    Dense(10, activation='softmax')   # 출력층
])

model.summary()

### (4) 모델 컴파일 (Compile)
모델을 훈련하기 전에, `compile` 메서드를 통해 학습 과정을 설정합니다.
- **Optimizer**: `adam` 옵티마이저를 사용합니다.
- **Loss**: `sparse_categorical_crossentropy`를 사용합니다. 타겟 레이블(y)이 원-핫 인코딩 형태가 아닌 정수 형태일 때 사용합니다.
- **Metrics**: 훈련 및 테스트 과정에서 모니터링할 지표로 `accuracy`(정확도)를 설정합니다.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### (5) 모델 훈련
`fit` 메서드를 사용하여 모델을 훈련합니다. 이 메서드는 훈련 데이터를 사용하여 모델의 가중치를 업데이트하고, `validation_data`로 지정된 검증 데이터로 매 에포크마다 성능을 평가합니다. 훈련 과정의 모든 정보는 `history` 객체에 저장됩니다.

In [ ]:
history = model.fit(X_train, y_train, 
                      batch_size=batch_size, 
                      epochs=epochs, 
                      validation_data=(X_test, y_test))

### (6) 모델 평가 및 학습 과정 시각화
`evaluate` 메서드로 테스트 데이터에 대한 최종 성능을 확인하고, `history` 객체를 사용하여 에포크별 손실 및 정확도 변화를 시각화하여 학습이 잘 이루어졌는지, 과적합은 없는지 등을 분석합니다.

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

# 학습 과정 시각화를 위한 데이터프레임 생성
history_df = pd.DataFrame(history.history)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 정확도 그래프
ax1.plot(history_df['accuracy'], label='Training Accuracy')
ax1.plot(history_df['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Training and Validation Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# 손실 그래프
ax2.plot(history_df['loss'], label='Training Loss')
ax2.plot(history_df['val_loss'], label='Validation Loss')
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.show()